# Import Padstack Definitions
This example shows how to import padstack definitions. This includes

- Download an example board
- Create a config file with hole information
- Create a config file with pad and anti-pad information

## Import the required packages

In [1]:
import json
from pathlib import Path
import tempfile

from IPython.display import display
from ansys.aedt.core.downloads import download_file
import pandas as pd

from pyedb import Edb

AEDT_VERSION = "2024.2"


Download the example PCB data.

In [2]:
temp_folder = tempfile.TemporaryDirectory(suffix=".ansys").name
file_edb = download_file(source="edb/ANSYS-HSD_V1.aedb", destination=temp_folder)

## Load example layout.

In [3]:
edbapp = Edb(file_edb, edbversion=AEDT_VERSION)

PyAEDT INFO: Logger is initialized in EDB.


PyAEDT INFO: legacy v0.37.0


PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]


PyAEDT INFO: Database ANSYS-HSD_V1.aedb Opened in 2024.2


PyAEDT INFO: Cell main Opened


PyAEDT INFO: Builder was initialized.


PyAEDT INFO: EDB initialized.


## Create a config file with hole information

Keywords

- **name**. Name of the padstack definition.
- **hole_plating_thickness**. Hole plating thickness.
- **hole_range**. Supported types are 'through', 'begin_on_upper_pad', 'end_on_lower_pad', 'upper_pad_to_lower_pad'.
- **hole_parameters**.
  - **shape**. Supported types are 'circle', 'square', 'rectangle'.
  - Other parameters are shape dependent.
    - When shape is 'circle', supported parameter si 'diameter'.
    - When shape is 'square', supported parameter is 'size'.
    - When shape is 'rectangle', supported parameters are 'x_size', 'y_size'.

In [4]:
cfg = dict()
cfg["padstacks"] = {}
cfg["padstacks"]["definitions"] = [
    {
        "name": "v35h15",
        "hole_plating_thickness": "25um",
        "material": "copper",
        "hole_range": "through",
        "hole_parameters": {
            "shape": "circle",
            "diameter": "0.15mm",
        },
    }
]

In [5]:
df = pd.DataFrame(data=cfg["padstacks"]["definitions"])
display(df)

,name,hole_plating_thickness,material,hole_range,hole_parameters
0,v35h15,25um,copper,through,"{'shape': 'circle', 'diameter': '0.15mm'}"


In [6]:
cfg_file_path = Path(temp_folder) / "cfg.json"
with open(cfg_file_path, "w") as f:
    json.dump(cfg, f, indent=4, ensure_ascii=False)

Load config file

In [7]:
edbapp.configuration.load(cfg_file_path, apply_file=True)

## Create a config file with pad information

Keywords

- **name**. Name of the padstack definition.
- **pad_parameters**.
  - **regular_pad**. List of pad definition per layer.
    - **layer_name**. Name of the layer.
    - **shape**. Supported types are 'circle', 'square', 'rectangle', 'oval', 'bullet'.
    - Other parameters are shape dependent.
      - When shape is 'circle', supported parameter si 'diameter'.
      - When shape is 'square', supported parameter is 'size'.
      - When shape is 'rectangle', supported parameters are 'x_size', 'y_size'.
      - When shape is 'oval', supported parameters are 'x_size', 'y_size', 'corner_radius'.
      - When shape is 'bullet', supported parameters are 'x_size', 'y_size', 'corner_radius'.

In [8]:
cfg = dict()
cfg["padstacks"] = {}
cfg["padstacks"]["definitions"] = [
    {
        "name": "v35h15",
        "pad_parameters": {
            "regular_pad": [
                {
                    "layer_name": "1_Top",
                    "shape": "circle",
                    "offset_x": "0.1mm",
                    "offset_y": "0.1mm",
                    "rotation": "0",
                    "diameter": "0.5mm",
                },
                {
                    "layer_name": "Inner1(GND1)",
                    "shape": "square",
                    "offset_x": "0.1mm",
                    "offset_y": "0.1mm",
                    "rotation": "45deg",
                    "size": "0.5mm",
                },
            ],
            "anti_pad": [{"layer_name": "1_Top", "shape": "circle", "diameter": "1mm"}],
        },
    }
]

In [9]:
df = pd.DataFrame(data=cfg["padstacks"]["definitions"][0]["pad_parameters"]["regular_pad"])
display(df)

,layer_name,shape,offset_x,offset_y,rotation,diameter,size
0,1_Top,circle,0.1mm,0.1mm,0,0.5mm,NaN
1,Inner1(GND1),square,0.1mm,0.1mm,45deg,NaN,0.5mm


In [10]:
df = pd.DataFrame(data=cfg["padstacks"]["definitions"][0]["pad_parameters"]["anti_pad"])
display(df)

,layer_name,shape,diameter
0,1_Top,circle,1mm


In [11]:
cfg_file_path = Path(temp_folder) / "cfg.json"
with open(cfg_file_path, "w") as f:
    json.dump(cfg, f, indent=4, ensure_ascii=False)

Load config file

In [12]:
edbapp.configuration.load(cfg_file_path, apply_file=True)

## Save and close Edb
The temporary folder will be deleted once the execution of this script is finished. Replace edbapp.save() with
edbapp.save_as("C:/example.aedb") to keep the example project.

In [13]:
edbapp.save()
edbapp.close()

True